In [1]:
import pandas as pd
import numpy as np

In [2]:
store_lists_3_DMAs=pd.read_excel("/home/jian/Projects/Big_Lots/New_TA/P_S_Selections/0531_Kayla_Request/SOTF_3DMAs_0531.xlsx",dtype=str)

In [3]:
store_lists_3_DMAs=store_lists_3_DMAs.fillna("nan")

In [4]:
Columns_SOTF_stores=[x for x in store_lists_3_DMAs['COLUMBUS, OH'] if x!="nan"]
AUSTIN_SOTF_stores=[x for x in store_lists_3_DMAs['AUSTIN'] if x!="nan"]
PHOENIX_SOTF_stores=[x for x in store_lists_3_DMAs['PHOENIX (PRESCOTT)'] if x!="nan"]

In [5]:
sales_zip_by_store=pd.read_csv("/home/jian/Projects/Big_Lots/New_TA/P_S_Selections/0531_Kayla_Request/sales_by_zip.csv",dtype=str)

In [6]:
sales_zip_by_store=sales_zip_by_store[['zip','revenue_flag','location_id']].drop_duplicates()

In [7]:
df_location_dma=pd.DataFrame(columns=["location_id","DMA"])
df_Columns=pd.DataFrame(columns=["location_id","DMA"])
df_Columns['location_id']=Columns_SOTF_stores
df_Columns['DMA']="COLUMBUS, OH"

df_AUSTIN=pd.DataFrame(columns=["location_id","DMA"])
df_AUSTIN['location_id']=AUSTIN_SOTF_stores
df_AUSTIN['DMA']="AUSTIN"

df_PHOENIX=pd.DataFrame(columns=["location_id","DMA"])
df_PHOENIX['location_id']=PHOENIX_SOTF_stores
df_PHOENIX['DMA']="PHOENIX (PRESCOTT)"

df_location_dma=df_Columns.append(df_AUSTIN).append(df_PHOENIX)
df_location_dma=df_location_dma.reset_index()
del df_location_dma['index']

In [8]:
sales_zip_by_store['location_id']=sales_zip_by_store['location_id'].apply(lambda x: x.replace(" ",""))
sales_zip_by_store['location_id']=sales_zip_by_store['location_id'].apply(lambda x: x.replace(" ",""))
sales_zip_by_store['location_id']=sales_zip_by_store['location_id'].apply(lambda x: x.replace(" ",""))

df_location_dma['location_id']=df_location_dma['location_id'].apply(lambda x: x.replace(" ",""))
df_location_dma['location_id']=df_location_dma['location_id'].apply(lambda x: x.replace(" ",""))
df_location_dma['location_id']=df_location_dma['location_id'].apply(lambda x: x.replace(" ",""))

In [9]:
sales_zip_by_store_P_S=sales_zip_by_store[sales_zip_by_store['location_id'].isin(df_location_dma['location_id'])]
sales_zip_by_store_P_S=sales_zip_by_store_P_S[sales_zip_by_store_P_S['revenue_flag'].isin(["P","S"])]
sales_zip_by_store_P_S=pd.merge(sales_zip_by_store_P_S,df_location_dma,on="location_id",how="left")
sales_zip_by_store_P_S=sales_zip_by_store_P_S.rename(columns={"DMA":"DMA_for_store"})

In [10]:
sales_zip_by_store_P_S.head(2)

,zip,revenue_flag,location_id,DMA_for_store
0,43232,P,1,"COLUMBUS, OH"
1,43209,P,1,"COLUMBUS, OH"


In [17]:
Nielsen_Zip_DMA=pd.read_excel("/home/jian/Projects/Big_Lots/New_TA/P_S_Selections/0531_Kayla_Request/Zips by DMA by County16-17 nielsen.xlsx",skiprows=1,dtype=str)
Nielsen_Zip_DMA=Nielsen_Zip_DMA.iloc[:,[0,2]].drop_duplicates()
Nielsen_Zip_DMA=Nielsen_Zip_DMA.rename(columns={"CODE":"zip_cd","NAME":"DMA"})
Nielsen_Zip_DMA=Nielsen_Zip_DMA[Nielsen_Zip_DMA['DMA'].isin(df_location_dma['DMA'])]
Nielsen_Zip_DMA_Blance=Nielsen_Zip_DMA[~Nielsen_Zip_DMA['zip_cd'].isin(sales_zip_by_store_P_S['zip'])]

In [32]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/New_TA/P_S_Selections/0531_Kayla_Request/BL_3 DMAs zips for SOTF_20180531.xlsx",engine="xlsxwriter")

In [33]:
sales_zip_by_store_P_S.to_excel(writer,"P_S_Zips_3 DMAs",index=False)
Nielsen_Zip_DMA_Blance.to_excel(writer,"P_S_Balance_3 DMAs",index=False)
writer.save()